In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from scipy.io import loadmat


In [11]:
data = loadmat('SpanishDomainF.mat')
SpanishDomainF = data['SpanishDomainF']

In [ ]:
s = SpanishDomainF[:, 0] #first column
t = SpanishDomainF[:, 1]  #second column
weights = SpanishDomainF[:, 2]  #third column
names = [str(i) for i in range(1, 31)]

G = nx.Graph()
for i in range(len(s)):
    source_node = names[int(s[i])-1]
    target_node = names[int(t[i]-1)]
    weight = weights[i]
    G.add_edge(source_node, target_node, weight = weight)
plt.figure(11, figsize = (12,8))
pos = nx.spring_layout(G, seed = 42)